<a href="https://colab.research.google.com/github/agentescrossmodais/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
import os
from google.colab import drive  # API nativa do Google Colab
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.model_selection as ms
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.decomposition as decomposition
import pingouin as pg
import io

# --- Instalação de Bibliotecas ---
def install_package(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} já está instalado.")
    except ImportError:
        print(f"Instalando {package_name}...")
        try:
            !pip install {package_name}
            __import__(package_name)
            print(f"{package_name} instalado com sucesso.")
        except Exception as e:
            print(f"Erro ao instalar {package_name}: {e}")

install_package('pingouin')

# --- Configurações e Constantes ---
DEFAULT_PALETTE = "viridis"
FIGSIZE = (12, 8)

sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 20,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# --- Funções Auxiliares ---

def mount_google_drive():
    """Monta o Google Drive usando a API nativa."""
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    """Retorna o caminho completo no Drive."""
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory_exists_on_drive(relative_path):
    """Cria diretório no Drive, se não existir."""
    drive_path = get_drive_path(relative_path)
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"Diretório criado: {drive_path}")
    return drive_path

def create_figure():
    """Cria uma figura."""
    return plt.figure(figsize=FIGSIZE)

def save_fig(fig, filename, drive_folder_path):
    """Salva a figura no Drive."""
    try:
        filepath = os.path.join(drive_folder_path, filename)
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        print(f"Gráfico salvo: {filepath}")
    except Exception as e:
        print(f"Erro ao salvar '{filename}': {e}")
    finally:
        plt.close(fig)

# --- Funções de Visualização Aprimoradas ---

def plot_boxplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota boxplot."""
    try:
        fig = create_figure()
        sns.boxplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        if hue:
            plt.legend(title=hue.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_boxplot: {e}")

def plot_violin(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota violin plot."""
    try:
        fig = create_figure()
        sns.violinplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE, split=(hue is not None))
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        if hue:
            plt.legend(title=hue.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_violin: {e}")

def plot_kde(df, col, title, filename, drive_folder_path, hue=None):
    """Plota KDE plot."""
    try:
        fig = create_figure()
        if hue:
            for level in df[hue].unique():
                sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True)
            plt.legend(title=hue.replace('_', ' '))
        else:
            sns.kdeplot(df[col], shade=True)
        plt.title(title)
        plt.xlabel(col.replace('_', ' '))
        plt.ylabel('Densidade')
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_kde: {e}")

def plot_line(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota gráfico de linha."""
    try:
        fig = create_figure()
        if hue:
            sns.lineplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
            plt.legend(title=hue.replace('_', ' '))
        else:
            sns.lineplot(x=x_col, y=y_col, data=df)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_line: {e}")

def plot_scatterplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None, style=None):
    """Plota gráfico de dispersão."""
    try:
        fig = create_figure()
        sns.scatterplot(x=x_col, y=y_col, data=df, hue=hue, style=style, palette=DEFAULT_PALETTE)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        if hue:
            plt.legend(title=hue.replace('_', ' '))
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro em plot_scatterplot: {e}")


# --- Funções de Análise ---

def carregar_dados(url):
    """Carrega os dados do Google Sheets (CSV) em um DataFrame Pandas."""
    try:
        df = pd.read_csv(url)
        print("Dados carregados com sucesso!")
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None

def pre_processar_dados(df):
    """
    Realiza o pré-processamento dos dados.
    """
    print("Pré-processamento dos dados...")
    # Converte nomes de colunas para minúsculas e substitui espaços por _
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    # Verifica e lida com valores ausentes
    for col in df.columns:
        if df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(df[col].mean())
                print(f"Valores ausentes na coluna '{col}' preenchidos com a média.")
            else:
                df[col] = df[col].fillna(df[col].mode()[0])
                print(f"Valores ausentes na coluna '{col}' preenchidos com a moda.")
    return df


def analisar_correlacoes(df, colunas_numericas, drive_folder_path):
    """
    Analisa correlações entre colunas numéricas e gera heatmap.
    """
    print("Analisando correlações...")
    if len(colunas_numericas) < 2:
        print("Não há colunas numéricas suficientes para análise de correlação.")
        return None

    try:
        correlacao = df[colunas_numericas].corr()
        print("\nMatriz de Correlação:\n", correlacao)

        fig = create_figure()
        sns.heatmap(correlacao, annot=True, cmap=DEFAULT_PALETTE, fmt=".2f",
                    cbar_kws={'label': 'Correlação'})
        plt.title('Matriz de Correlação')
        save_fig(fig, 'matriz_correlacao.png', drive_folder_path)
        return correlacao

    except Exception as e:
        print(f"Erro na análise de correlação: {e}")
        return None

def aplicar_analise_fatorial(df, colunas_fatoriais, drive_folder_path):
    """Aplica análise fatorial para identificar dimensões subjacentes."""
    print("\n--- Análise Fatorial ---")

    if not all(col in df.columns for col in colunas_fatoriais):
        print("Algumas colunas para análise fatorial não foram encontradas.")
        return None

    try:
        dados_fatoriais = df[colunas_fatoriais].copy()

        # Padronizar os dados
        dados_fatoriais_std = (dados_fatoriais - dados_fatoriais.mean()) / dados_fatoriais.std()

        # Aplicar a análise fatorial
        fa = decomposition.FactorAnalysis(n_components=3, random_state=42)
        fa.fit(dados_fatoriais_std)

        # Imprimir as cargas fatoriais
        cargas_fatoriais = pd.DataFrame(fa.components_.T, index=colunas_fatoriais,
                                       columns=['Fator 1', 'Fator 2', 'Fator 3'])
        print("\nCargas Fatoriais:\n", cargas_fatoriais)

        # Interpretar os fatores (com base nas cargas)
        display(Markdown("""
        **Interpretação (Simulada):**

        *   **Fator 1:** Pode representar o "Desempenho Acadêmico".
        *   **Fator 2:** Pode representar o "Engajamento Escolar".
        *   **Fator 3:** Pode representar o "Bem-Estar Socioemocional".
        """))

        # Calcular os scores fatoriais e adicionar ao DataFrame
        scores_fatoriais = fa.transform(dados_fatoriais_std)
        df['fator_1'] = scores_fatoriais[:, 0]
        df['fator_2'] = scores_fatoriais[:, 1]
        df['fator_3'] = scores_fatoriais[:, 2]

        return df

    except Exception as e:
        print(f"Erro na análise fatorial: {e}")
        return None


def aplicar_analise_cluster(df, colunas_cluster, drive_folder_path):
    """Aplica análise de cluster (k-means) para identificar grupos."""
    print("\n--- Análise de Cluster (K-Means) ---")

    if not all(col in df.columns for col in colunas_cluster):
        print("Algumas colunas para análise de cluster não foram encontradas.")
        return None

    try:
        dados_cluster = df[colunas_cluster].copy()

        # Aplicar o k-means
        kmeans = cluster.KMeans(n_clusters=4, random_state=42, n_init=10)
        df['cluster'] = kmeans.fit_predict(dados_cluster)

        # Visualizar a distribuição dos clusters
        print("\nDistribuição dos Clusters:\n", df['cluster'].value_counts())

        # Visualizar características dos clusters (médias)
        print("\nCaracterísticas dos Clusters (Médias):\n", df.groupby('cluster')[colunas_cluster].mean())

        # Visualização 2D
        plot_scatterplot(df, colunas_cluster[0], colunas_cluster[1], 'Clusters de Alunos (K-Means)',
                         'clusters_kmeans.png', drive_folder_path, hue='cluster')

        return df

    except Exception as e:
        print(f"Erro na análise de cluster: {e}")
        return None

def realizar_testes_t(df, variavel_teste, grupo_coluna, drive_folder_path):
    """Realiza testes t independentes entre grupos e gera visualizações."""
    print(f"\n--- Testes t para {variavel_teste} ---")

    if grupo_coluna not in df.columns or variavel_teste not in df.columns:
        print(f"Colunas '{grupo_coluna}' ou '{variavel_teste}' não encontradas.")
        return

    grupos = df[grupo_coluna].unique()
    if len(grupos) < 2:
        print("Número insuficiente de grupos para o teste t.")
        return

    try:
        if not pd.api.types.is_numeric_dtype(df[variavel_teste]):
            df[variavel_teste] = pd.to_numeric(df[variavel_teste], errors='coerce')
            df.dropna(subset=[variavel_teste], inplace=True)

        grupo1 = df[df[grupo_coluna] == grupos[0]][variavel_teste]
        grupo2 = df[df[grupo_coluna] == grupos[1]][variavel_teste]

        ttest_result = pg.ttest(grupo1, grupo2, paired=False)
        print(f"\nTeste t para {variavel_teste} ({grupos[0]} vs {grupos[1]}):\n", ttest_result)

        plot_boxplot(df, grupo_coluna, variavel_teste, f"{variavel_teste} - Boxplot",
                     f"{variavel_teste}_boxplot.png", drive_folder_path, hue=grupo_coluna)
        plot_violin(df, grupo_coluna, variavel_teste, f"{variavel_teste} - Violin Plot",
                    f"{variavel_teste}_violin.png", drive_folder_path, hue=grupo_coluna)
        plot_kde(df, variavel_teste, f"{variavel_teste} - KDE", f"{variavel_teste}_kde.png",
                 drive_folder_path, hue=grupo_coluna)

    except Exception as e:
        print(f"Erro no teste t para '{variavel_teste}': {e}")



def gerar_dados_teste(n_alunos=40):
    """Gera dados sintéticos para teste."""
    np.random.seed(42)

    dados = pd.DataFrame({
        'aluno_id': range(1, n_alunos + 1),
        'desempenho_prova': np.random.normal(loc=70, scale=15, size=n_alunos),
        'frequencia': np.random.uniform(low=0.7, high=1, size=n_alunos),
        'participacao_atividades': np.random.poisson(lam=3, size=n_alunos),
        'questionario_socioemocional': np.random.randint(1, 6, size=n_alunos),
        'feedback_professor': np.random.normal(loc=4, scale=0.8, size=n_alunos),
        'grupo': np.random.choice(['A', 'B'], size=n_alunos)
    })
    return dados

def gerar_relatorio_estatistico(df, drive_folder_path):
    """
    Gera um relatório estatístico em formato de texto e salva em um arquivo .txt.
    """
    print("Gerando relatório estatístico...")

    try:
        # Usa io.StringIO para construir o relatório em memória
        relatorio_buffer = io.StringIO()

        # Informações básicas
        relatorio_buffer.write("# Relatório Estatístico - Análise de Dados\n\n")
        relatorio_buffer.write("## Informações do DataFrame\n")
        relatorio_buffer.write(f"Número de linhas: {len(df)}\n")
        relatorio_buffer.write(f"Número de colunas: {len(df.columns)}\n\n")
        relatorio_buffer.write("### Tipos de Dados\n")
        relatorio_buffer.write(str(df.dtypes) + "\n\n")
        relatorio_buffer.write("### Primeiras 5 linhas\n")
        relatorio_buffer.write(str(df.head()) + "\n\n")
        relatorio_buffer.write("### Estatísticas Descritivas\n")
        relatorio_buffer.write(str(df.describe(include='all')) + "\n\n")

        # Análise de correlação (se aplicável)
        colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
        if len(colunas_numericas) >= 2:
            relatorio_buffer.write("## Matriz de Correlação\n")
            relatorio_buffer.write(str(df[colunas_numericas].corr()) + "\n\n")

        # Análise fatorial (se aplicável)
        if 'fator_1' in df.columns:
            relatorio_buffer.write("## Cargas Fatoriais (Análise Fatorial)\n")
            # Supondo que você já rodou a análise fatorial e tem as cargas
            colunas_fatoriais = ['desempenho_prova', 'frequencia', 'participacao_atividades',
                                 'questionario_socioemocional', 'feedback_professor']
            if all(col in df.columns for col in colunas_fatoriais):
                dados_fatoriais = df[colunas_fatoriais].copy()
                dados_fatoriais_std = (dados_fatoriais - dados_fatoriais.mean()) / dados_fatoriais.std()
                fa = decomposition.FactorAnalysis(n_components=3, random_state=42)
                fa.fit(dados_fatoriais_std)
                cargas_fatoriais = pd.DataFrame(fa.components_.T, index=colunas_fatoriais,
                                               columns=['Fator 1', 'Fator 2', 'Fator 3'])
                relatorio_buffer.write(str(cargas_fatoriais) + "\n\n")

        # Análise de cluster (se aplicável)
        if 'cluster' in df.columns:
            relatorio_buffer.write("## Análise de Cluster\n")
            relatorio_buffer.write("### Distribuição dos Clusters\n")
            relatorio_buffer.write(str(df['cluster'].value_counts()) + "\n\n")
            relatorio_buffer.write("### Características dos Clusters (Médias)\n")
            relatorio_buffer.write(str(df.groupby('cluster')[['fator_1', 'fator_2', 'fator_3']].mean()) + "\n\n")

        # Testes t (se aplicável)
        grupo_coluna = 'grupo'
        if grupo_coluna in df.columns:
            relatorio_buffer.write("## Testes t\n")
            for col in colunas_numericas:
                if col not in ['fator_1', 'fator_2', 'fator_3', 'cluster', 'aluno_id']:  # Evita fatores e ID
                    grupos = df[grupo_coluna].unique()
                    if len(grupos) >= 2:
                        if not pd.api.types.is_numeric_dtype(df[col]):
                            df[col] = pd.to_numeric(df[col], errors='coerce')
                            df.dropna(subset=[col], inplace=True)
                        grupo1 = df[df[grupo_coluna] == grupos[0]][col]
                        grupo2 = df[df[grupo_coluna] == grupos[1]][col]
                        ttest_result = pg.ttest(grupo1, grupo2, paired=False)
                        relatorio_buffer.write(f"### Teste t para {col} ({grupos[0]} vs {grupos[1]}):\n")
                        relatorio_buffer.write(str(ttest_result) + "\n\n")

        # Salva o relatório em um arquivo .txt no Drive
        filepath = os.path.join(drive_folder_path, "sumario.txt")
        with open(filepath, "w") as f:
            f.write(relatorio_buffer.getvalue())
        print(f"Relatório estatístico salvo em: {filepath}")

    except Exception as e:
        print(f"Erro ao gerar o relatório estatístico: {e}")


# --- Função Principal (main) ---
if __name__ == "__main__":
    mount_google_drive()
    drive_base_path = '/content/drive/MyDrive/'
    graficos_drive_path = ensure_directory_exists_on_drive('graficos')
    sumario_drive_path = ensure_directory_exists_on_drive('sumario')


    display(Markdown("# Análise do Sistema de Indicadores Sintéticos (Agentes Crossmodais)"))

    # --- Carregar Dados ---
    # Usar os dados de teste gerados:
    df = gerar_dados_teste()

    # --- Pré-processar Dados ---
    df = pre_processar_dados(df)

    # --- Análise Exploratória e Relatório ---
    gerar_relatorio_estatistico(df, sumario_drive_path)


    # --- Definir colunas ---
    colunas_numericas = df.select_dtypes(include=np.number).columns.tolist()
    colunas_fatoriais = ['desempenho_prova', 'frequencia', 'participacao_atividades', 'questionario_socioemocional', 'feedback_professor']
    colunas_cluster = ['fator_1', 'fator_2', 'fator_3']
    grupo_coluna = 'grupo'

    # --- Análises ---
    # 1. Análise de Correlações
    analisar_correlacoes(df, colunas_numericas, graficos_drive_path)

    # 2. Análise Fatorial
    df = aplicar_analise_fatorial(df, colunas_fatoriais, graficos_drive_path)

    # 3. Análise de Cluster
    if all(col in df.columns for col in ['fator_1', 'fator_2', 'fator_3']):
        df = aplicar_analise_cluster(df, colunas_cluster, graficos_drive_path)

    # 4. Testes t
    if grupo_coluna in df.columns:
      for col in colunas_numericas:
          if col not in ['fator_1', 'fator_2', 'fator_3', 'cluster', 'aluno_id']: # Evita colunas geradas
            realizar_testes_t(df, col, grupo_coluna, graficos_drive_path)


    display(Markdown("## Conclusões"))
    display(Markdown("Resultados da análise do sistema de indicadores sintéticos."))

pingouin já está instalado.
Mounted at /content/drive
Diretório criado: /content/drive/MyDrive/graficos
Diretório criado: /content/drive/MyDrive/sumario


# Análise do Sistema de Indicadores Sintéticos (Agentes Crossmodais)

Pré-processamento dos dados...
Gerando relatório estatístico...
Relatório estatístico salvo em: /content/drive/MyDrive/sumario/sumario.txt
Analisando correlações...

Matriz de Correlação:
                              aluno_id  desempenho_prova  frequencia  \
aluno_id                     1.000000         -0.250444   -0.112745   
desempenho_prova            -0.250444          1.000000    0.057739   
frequencia                  -0.112745          0.057739    1.000000   
participacao_atividades      0.014239          0.278820    0.011033   
questionario_socioemocional  0.171382         -0.002671    0.060004   
feedback_professor          -0.160473          0.411418   -0.021641   

                             participacao_atividades  \
aluno_id                                    0.014239   
desempenho_prova                            0.278820   
frequencia                                  0.011033   
participacao_atividades                     1.000000   
questionario_socioemocional      


        **Interpretação (Simulada):**

        *   **Fator 1:** Pode representar o "Desempenho Acadêmico".
        *   **Fator 2:** Pode representar o "Engajamento Escolar".
        *   **Fator 3:** Pode representar o "Bem-Estar Socioemocional".
        


--- Análise de Cluster (K-Means) ---

Distribuição dos Clusters:
 cluster
0    14
2    12
3     8
1     6
Name: count, dtype: int64

Características dos Clusters (Médias):
           fator_1   fator_2   fator_3
cluster                              
0       -0.562797 -0.336891 -0.087074
1        1.145879  0.805695 -0.120058
2        0.601587 -0.485806  0.067105
3       -0.776896  0.713998  0.141765
Gráfico salvo: /content/drive/MyDrive/graficos/clusters_kmeans.png

--- Testes t para desempenho_prova ---

Teste t para desempenho_prova (B vs A):
                T        dof alternative     p-val          CI95%   cohen-d  \
T-test  0.300078  37.904673   two-sided  0.765757  [-7.8, 10.52]  0.093871   

         BF10     power  
T-test  0.321  0.059553  


...

Gráfico salvo: /content/drive/MyDrive/graficos/feedback_professor_violin.png


<ipython-input-3-097bcdbfa999>:128: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True)
<ipython-input-3-097bcdbfa999>:128: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(df.loc[df[hue] == level, col], label=level, shade=True)


Gráfico salvo: /content/drive/MyDrive/graficos/feedback_professor_kde.png


## Conclusões

Resultados da análise do sistema de indicadores sintéticos.